<a href="https://colab.research.google.com/github/LaputMarkDanielle/CPE-201L-DSA-2-A/blob/main/Progress_Report_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Universal Task Scheduler (single-file)
- Kivy UI
- SQLite local storage (reminders.db)
- Date/time picker popup
- Swipe gestures (left snooze, right done)
- Snooze, undo, repeat, history
- Per-task and global custom sounds (upload/preview/reset)
- Settings overlay slides from right
- Android-compatible plyer usage for notification and vibration
"""


import os
import sqlite3
import threading
from datetime import datetime, timedelta
from functools import partial


from kivy.app import App
from kivy.clock import Clock
from kivy.core.window import Window
from kivy.core.audio import SoundLoader
from kivy.properties import StringProperty, NumericProperty, BooleanProperty
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.gridlayout import GridLayout
from kivy.uix.scrollview import ScrollView
from kivy.uix.label import Label
from kivy.uix.button import Button
from kivy.uix.popup import Popup
from kivy.uix.spinner import Spinner
from kivy.uix.textinput import TextInput
from kivy.uix.filechooser import FileChooserIconView
from kivy.animation import Animation


# plyer features (notification, vibrator, filechooser)
from plyer import notification, vibrator, filechooser


# Constants
DB_FILE = "reminders.db"
SOUNDS_DIR = "sounds"
UNDO_DEFAULT = 10
HISTORY_CLEAN_DAYS = 30
CHECK_INTERVAL_SECONDS = 20  # how often scheduler checks due tasks


# Ensure sounds directory
os.makedirs(SOUNDS_DIR, exist_ok=True)




# ---------------------------
# Database helpers
# ---------------------------
def init_db():
   conn = sqlite3.connect(DB_FILE)
   c = conn.cursor()
   # tasks table
   c.execute("""
       CREATE TABLE IF NOT EXISTS tasks (
           id INTEGER PRIMARY KEY AUTOINCREMENT,
           title TEXT NOT NULL,
           created_iso TEXT NOT NULL,
           due_iso TEXT,
           priority TEXT DEFAULT 'Normal',
           alarm_type TEXT DEFAULT 'notify_ding',
           sound_path TEXT,
           repeat_mode TEXT DEFAULT 'none',
           repeat_days TEXT,
           completed_iso TEXT,
           cancelled INTEGER DEFAULT 0
       )
   """)
   # settings table
   c.execute("""
       CREATE TABLE IF NOT EXISTS settings (
           key TEXT PRIMARY KEY,
           value TEXT
       )
   """)
   # default settings
   defaults = {
       "global_default_sound": "",
       "default_alarm": "notify_ding",
       "default_snooze": "5",
       "theme_mode": "light",
       "undo_seconds": str(UNDO_DEFAULT)
   }
   for k, v in defaults.items():
       c.execute("INSERT OR IGNORE INTO settings (key, value) VALUES (?, ?)", (k, v))
   conn.commit()
   conn.close()


   # cleanup old history
   cleanup_history(HISTORY_CLEAN_DAYS)




def db_get_setting(key):
   conn = sqlite3.connect(DB_FILE)
   c = conn.cursor()
   c.execute("SELECT value FROM settings WHERE key=?", (key,))
   r = c.fetchone()
   conn.close()
   return r[0] if r else None




def db_set_setting(key, value):
   conn = sqlite3.connect(DB_FILE)
   c = conn.cursor()
   c.execute("INSERT OR REPLACE INTO settings (key, value) VALUES (?, ?)", (key, str(value)))
   conn.commit()
   conn.close()




def cleanup_history(days=30):
   cutoff = datetime.utcnow() - timedelta(days=days)
   cutoff_iso = cutoff.isoformat()
   conn = sqlite3.connect(DB_FILE)
   c = conn.cursor()
   try:
       c.execute("DELETE FROM tasks WHERE completed_iso IS NOT NULL AND completed_iso < ?", (cutoff_iso,))
       conn.commit()
   except Exception as e:
       print("cleanup_history error:", e)
   conn.close()




# ---------------------------
# Sound manager
# ---------------------------
class SoundPlayer:
   @staticmethod
   def play(path):
       if not path:
           # nothing to play
           return
       try:
           # use a thread so playback doesn't block UI
           def runner(p):
               try:
                   s = SoundLoader.load(p)
                   if s:
                       s.play()
               except Exception as e:
                   print("Sound play error:", e)
           t = threading.Thread(target=runner, args=(path,), daemon=True)
           t.start()
       except Exception as e:
           print("SoundPlayer error:", e)




# ---------------------------
# Small helpers
# ---------------------------
def iso_now():
   return datetime.utcnow().isoformat()




def parse_iso(s):
   try:
       return datetime.fromisoformat(s)
   except Exception:
       return None




def friendly_dt(iso):
   dt = parse_iso(iso)
   if not dt:
       return iso or ""
   return dt.strftime("%Y-%m-%d %H:%M")




# ---------------------------
# Task item (with swipe)
# ---------------------------
class TaskItem(BoxLayout):
   task_id = NumericProperty(0)
   title = StringProperty("")
   due_iso = StringProperty("")
   priority = StringProperty("Normal")
   completed = BooleanProperty(False)


   def __init__(self, row, manager_ref, **kwargs):
       super().__init__(orientation="horizontal", size_hint_y=None, height=64, padding=6, **kwargs)
       self.manager = manager_ref
       self.task_id = row.get("id")
       self.title = row.get("title")
       self.due_iso = row.get("due_iso") or ""
       self.priority = row.get("priority") or "Normal"
       self.alarm_type = row.get("alarm_type")
       self.sound_path = row.get("sound_path")
       self.repeat_mode = row.get("repeat_mode")
       self.repeat_days = row.get("repeat_days")
       self.completed = bool(row.get("completed_iso"))
       self.cancelled = bool(row.get("cancelled"))


       # label area
       left = BoxLayout(orientation="vertical")
       pr_symbol = "🔴 " if self.priority == "High" else ""
       left.add_widget(Label(text=f"{pr_symbol}{self.title}", halign="left", valign="middle"))
       left.add_widget(Label(text=f"Due: {friendly_dt(self.due_iso)}", font_size=12, halign="left"))
       self.add_widget(left)


       # action buttons
       btns = BoxLayout(size_hint_x=None, width=260, spacing=6)
       edit_btn = Button(text="Edit", size_hint_x=None, width=70)
       edit_btn.bind(on_press=self.on_edit)
       btns.add_widget(edit_btn)


       done_btn = Button(text="Done", size_hint_x=None, width=60)
       done_btn.bind(on_press=self.on_complete)
       btns.add_widget(done_btn)


       snooze_btn = Button(text="Snooze", size_hint_x=None, width=80)
       snooze_btn.bind(on_press=lambda *a: self.manager.snooze_task(self.task_id, int(db_get_setting("default_snooze") or 5)))
       btns.add_widget(snooze_btn)


       cancel_btn = Button(text="Cancel", size_hint_x=None, width=60, background_color=(1, .3, .3, 1))
       cancel_btn.bind(on_press=self.on_cancel)
       btns.add_widget(cancel_btn)


       self.add_widget(btns)


       self._touch_start_x = None


   def on_edit(self, *a):
       self.manager.open_task_popup(mode="edit", task_id=self.task_id)


   def on_complete(self, *a):
       self.manager.complete_task(self.task_id)


   def on_cancel(self, *a):
       self.manager.confirm_cancel(self.task_id)


   def on_touch_down(self, touch):
       if self.collide_point(*touch.pos):
           self._touch_start_x = touch.x
       return super().on_touch_down(touch)


   def on_touch_up(self, touch):
       if self._touch_start_x is None:
           return super().on_touch_up(touch)
       dx = touch.x - self._touch_start_x
       self._touch_start_x = None
       if abs(dx) < 50:
           return super().on_touch_up(touch)
       if dx < 0:
           # swipe left -> snooze
           try:
               snooze_min = int(db_get_setting("default_snooze") or 5)
               self.manager.snooze_task(self.task_id, snooze_min)
           except Exception as e:
               print("snooze error:", e)
       else:
           # swipe right -> complete
           self.manager.complete_task(self.task_id)
       return True




# ---------------------------
# Add/Edit Task Popup
# ---------------------------
class AddEditPopup(Popup):
   def __init__(self, manager_ref, mode="add", task_data=None, **kwargs):
       title = "Add Task" if mode == "add" else "Edit Task" if mode == "edit" else "Do Again"
       super().__init__(title=title, size_hint=(0.96, 0.9), **kwargs)
       self.manager = manager_ref
       self.mode = mode
       self.task_data = task_data or {}
       layout = BoxLayout(orientation="vertical", spacing=8, padding=8)


       # Title
       self.title_input = TextInput(text=self.task_data.get("title", ""), size_hint_y=None, height=40, hint_text="Task title")
       layout.add_widget(self.title_input)


       # Date/time spinners
       dt_box = BoxLayout(size_hint_y=None, height=40)
       now = datetime.now()
       years = [str(y) for y in range(now.year, now.year + 3)]
       months = [str(m).zfill(2) for m in range(1, 13)]
       days = [str(d).zfill(2) for d in range(1, 32)]
       self.year = Spinner(text=str(now.year), values=years, size_hint_x=None, width=100)
       self.month = Spinner(text=str(now.month).zfill(2), values=months, size_hint_x=None, width=80)
       self.day = Spinner(text=str(now.day).zfill(2), values=days, size_hint_x=None, width=80)
       hours = [str(h).zfill(2) for h in range(0, 24)]
       mins = [str(m).zfill(2) for m in range(0, 60, 5)]
       self.hour = Spinner(text=str(now.hour).zfill(2), values=hours, size_hint_x=None, width=80)
       self.minute = Spinner(text=str((now.minute//5)*5).zfill(2), values=mins, size_hint_x=None, width=80)
       dt_box.add_widget(self.year); dt_box.add_widget(self.month); dt_box.add_widget(self.day)
       dt_box.add_widget(self.hour); dt_box.add_widget(self.minute)
       layout.add_widget(dt_box)


       # Priority
       prio_box = BoxLayout(size_hint_y=None, height=40)
       self.prio_spinner = Spinner(text=self.task_data.get("priority", db_get_setting("default_priority") or "Normal"), values=["Normal", "High"], size_hint_x=None, width=140)
       prio_box.add_widget(Label(text="Priority:", size_hint_x=None, width=80))
       prio_box.add_widget(self.prio_spinner)
       layout.add_widget(prio_box)


       # Alarm type
       alarm_box = BoxLayout(size_hint_y=None, height=40)
       self.alarm_spinner = Spinner(text=self.task_data.get("alarm_type", db_get_setting("default_alarm") or "notify_ding"),
                                    values=["notify_ding", "ring_vibrate", "vibrate_notify"], size_hint_x=None, width=220)
       alarm_box.add_widget(Label(text="Alarm:", size_hint_x=None, width=80))
       alarm_box.add_widget(self.alarm_spinner)
       layout.add_widget(alarm_box)


       # Sound upload + preview + reset
       sound_box = BoxLayout(size_hint_y=None, height=40)
       self.sound_label = Label(text=os.path.basename(self.task_data.get("sound_path")) if self.task_data.get("sound_path") else "Default (ding)", halign="left")
       sound_box.add_widget(self.sound_label)
       upload_btn = Button(text="Upload Sound", size_hint_x=None, width=120)
       upload_btn.bind(on_press=self.open_filechooser)
       sound_box.add_widget(upload_btn)
       preview_btn = Button(text="Preview", size_hint_x=None, width=80)
       preview_btn.bind(on_press=self.preview_sound)
       sound_box.add_widget(preview_btn)
       reset_btn = Button(text="Reset", size_hint_x=None, width=80)
       reset_btn.bind(on_press=self.reset_to_default_sound)
       sound_box.add_widget(reset_btn)
       layout.add_widget(sound_box)


       # Repeat
       repeat_box = BoxLayout(size_hint_y=None, height=40)
       self.repeat_spinner = Spinner(text=self.task_data.get("repeat_mode", "none"), values=["none", "once", "daily", "weekly", "forever"], size_hint_x=None, width=140)
       repeat_box.add_widget(Label(text="Repeat:", size_hint_x=None, width=80))
       repeat_box.add_widget(self.repeat_spinner)
       layout.add_widget(repeat_box)


       # Weekdays toggles for weekly repeat
       wd_box = BoxLayout(size_hint_y=None, height=40)
       self.weekday_buttons = []
       for i, wd in enumerate(["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]):
           b = Button(text=wd, size_hint_x=None, width=44)
           b.background_color = (0.85, 0.85, 0.85, 1)
           b.bind(on_press=partial(self.toggle_weekday, i))
           self.weekday_buttons.append(b)
           wd_box.add_widget(b)
       layout.add_widget(wd_box)


       # Footer (cancel/save)
       footer = BoxLayout(size_hint_y=None, height=50)
       cancel_btn = Button(text="Cancel")
       cancel_btn.bind(on_press=self.dismiss)
       save_btn = Button(text="Save", background_color=(0.2, 0.7, 0.2, 1))
       save_btn.bind(on_press=self.on_save)
       footer.add_widget(cancel_btn)
       footer.add_widget(save_btn)
       layout.add_widget(footer)


       # populate fields if editing / doagain
       if self.mode in ("edit", "doagain") and self.task_data:
           try:
               dt = parse_iso(self.task_data.get("due_iso"))
               if dt:
                   self.year.text = str(dt.year)
                   self.month.text = str(dt.month).zfill(2)
                   self.day.text = str(dt.day).zfill(2)
                   self.hour.text = str(dt.hour).zfill(2)
                   self.minute.text = str(dt.minute).zfill(2)
           except Exception:
               pass
           self.title_input.text = self.task_data.get("title", "")
           self.prio_spinner.text = self.task_data.get("priority", "Normal")
           self.alarm_spinner.text = self.task_data.get("alarm_type", db_get_setting("default_alarm") or "notify_ding")
           sp = self.task_data.get("sound_path")
           if sp:
               self.sound_label.text = os.path.basename(sp)
               self._selected_sound = sp
           else:
               self.sound_label.text = "Default (ding)"
               self._selected_sound = None
           self.repeat_spinner.text = self.task_data.get("repeat_mode", "none")
           if self.task_data.get("repeat_days"):
               for idx_str in (self.task_data.get("repeat_days") or "").split(","):
                   if idx_str.strip().isdigit():
                       idx = int(idx_str)
                       if 0 <= idx < 7:
                           self.weekday_buttons[idx].background_color = (0.45, 0.85, 0.45, 1)
       else:
           self._selected_sound = None


       self.content = layout
       self.filechooser_popup = None


   def toggle_weekday(self, idx, *a):
       b = self.weekday_buttons[idx]
       if b.background_color[0] > 0.7:
           b.background_color = (0.45, 0.85, 0.45, 1)
       else:
           b.background_color = (0.85, 0.85, 0.85, 1)


   def open_filechooser(self, *a):
       # Try using plyer.filechooser first (Android-friendly)
       try:
           selection = filechooser.open_file(title="Select sound file", filters=["*.wav", "*.mp3"])
           if selection and len(selection) > 0:
               src = selection[0]
               basename = f"{int(datetime.utcnow().timestamp())}_{os.path.basename(src)}"
               dest = os.path.join(SOUNDS_DIR, basename)
               try:
                   with open(src, "rb") as rf, open(dest, "wb") as wf:
                       wf.write(rf.read())
                   self._selected_sound = dest
                   self.sound_label.text = os.path.basename(dest)
               except Exception as e:
                   print("Copy error:", e)
       except Exception:
           # fallback to Kivy filechooser popup
           if not self.filechooser_popup:
               fc = FileChooserIconView(path=".", filters=["*.mp3", "*.wav"])
               ok = Button(text="Select", size_hint_y=None, height=40)
               box = BoxLayout(orientation="vertical")
               box.add_widget(fc)
               row = BoxLayout(size_hint_y=None, height=40)
               row.add_widget(ok)
               close = Button(text="Close")
               row.add_widget(close)
               box.add_widget(row)
               popup = Popup(title="Choose a sound file", content=box, size_hint=(0.9, 0.9))
               ok.bind(on_press=lambda inst: self.select_file(fc.path, fc.selection, popup))
               close.bind(on_press=popup.dismiss)
               self.filechooser_popup = popup
           self.filechooser_popup.open()


   def select_file(self, path, selection, popup):
       if not selection:
           return
       src = selection[0]
       basename = f"{int(datetime.utcnow().timestamp())}_{os.path.basename(src)}"
       dest = os.path.join(SOUNDS_DIR, basename)
       try:
           with open(src, "rb") as rf, open(dest, "wb") as wf:
               wf.write(rf.read())
           self._selected_sound = dest
           self.sound_label.text = os.path.basename(dest)
           popup.dismiss()
       except Exception as e:
           print("File copy error:", e)


   def preview_sound(self, *a):
       path = getattr(self, "_selected_sound", None)
       if not path:
           global_def = db_get_setting("global_default_sound") or ""
           if global_def:
               SoundPlayer.play(global_def)
           else:
               notification.notify(title="Preview", message="Default ding (system)")
       else:
           SoundPlayer.play(path)


   def reset_to_default_sound(self, *a):
       self._selected_sound = None
       self.sound_label.text = "Default (ding)"


   def on_save(self, *a):
       # build due iso
       try:
           yr = int(self.year.text); mo = int(self.month.text); da = int(self.day.text)
           hr = int(self.hour.text); mn = int(self.minute.text)
           due = datetime(year=yr, month=mo, day=da, hour=hr, minute=mn)
           due_iso = due.isoformat()
       except Exception:
           notification.notify(title="Error", message="Invalid date/time")
           return
       title = self.title_input.text.strip()
       if not title:
           notification.notify(title="Error", message="Please enter a title")
           return
       pr = self.prio_spinner.text
       alarm = self.alarm_spinner.text
       soundp = getattr(self, "_selected_sound", None)
       repeat_mode = self.repeat_spinner.text
       repeat_days = ",".join(str(i) for i,b in enumerate(self.weekday_buttons) if b.background_color[0] < 0.7)


       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       if self.mode == "edit" and self.task_data and self.task_data.get("id"):
           c.execute("""UPDATE tasks SET title=?, due_iso=?, priority=?, alarm_type=?, sound_path=?, repeat_mode=?, repeat_days=? WHERE id=?""",
                     (title, due_iso, pr, alarm, soundp, repeat_mode, repeat_days, self.task_data["id"]))
       else:
           created = iso_now()
           c.execute("""INSERT INTO tasks (title, created_iso, due_iso, priority, alarm_type, sound_path, repeat_mode, repeat_days)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
                     (title, created, due_iso, pr, alarm, soundp, repeat_mode, repeat_days))
       conn.commit()
       conn.close()
       self.manager.reload_tasks_ui()
       self.dismiss()




# ---------------------------
# Settings overlay (slides from right)
# ---------------------------
class SettingsOverlay(BoxLayout):
   def __init__(self, manager_ref, **kwargs):
       super().__init__(orientation="vertical", size_hint=(0.8, 1), **kwargs)
       self.manager = manager_ref
       self.padding = 10
       self.spacing = 8
       self.add_widget(Label(text="Settings", size_hint_y=None, height=40))
       # theme toggle
       theme_box = BoxLayout(size_hint_y=None, height=40)
       theme_box.add_widget(Label(text="Theme:", size_hint_x=None, width=80))
       self.theme_btn = Button(text="Toggle Light/Dark")
       self.theme_btn.bind(on_press=self.toggle_theme)
       theme_box.add_widget(self.theme_btn)
       self.add_widget(theme_box)
       # snooze default
       snooze_box = BoxLayout(size_hint_y=None, height=40)
       snooze_box.add_widget(Label(text="Default Snooze (min):", size_hint_x=None, width=150))
       self.snooze_spinner = Spinner(text=db_get_setting("default_snooze") or "5", values=[str(v) for v in (5,10,15,20)], size_hint_x=None, width=80)
       self.snooze_spinner.bind(text=self.on_snooze_change)
       snooze_box.add_widget(self.snooze_spinner)
       self.add_widget(snooze_box)
       # default alarm
       alarm_box = BoxLayout(size_hint_y=None, height=40)
       alarm_box.add_widget(Label(text="Default Alarm:", size_hint_x=None, width=120))
       self.alarm_spinner = Spinner(text=db_get_setting("default_alarm") or "notify_ding",
                                    values=["notify_ding","ring_vibrate","vibrate_notify"], size_hint_x=None, width=180)
       self.alarm_spinner.bind(text=self.on_default_alarm_change)
       alarm_box.add_widget(self.alarm_spinner)
       self.add_widget(alarm_box)
       # sounds management
       self.add_widget(Label(text="Uploaded Sounds:", size_hint_y=None, height=30))
       self.sounds_list = BoxLayout(orientation="vertical")
       self.refresh_sounds_list()
       self.add_widget(self.sounds_list)
       upload_global = Button(text="Upload Default Sound")
       upload_global.bind(on_press=manager_ref.open_upload_default_sound)
       self.add_widget(upload_global)
       reset_all_btn = Button(text="Reset All Task Sounds to Ding")
       reset_all_btn.bind(on_press=self.reset_all_task_sounds)
       self.add_widget(reset_all_btn)
       close = Button(text="Close", size_hint_y=None, height=40)
       close.bind(on_press=manager_ref.toggle_settings_overlay)
       self.add_widget(close)


   def toggle_theme(self, *a):
       mode = db_get_setting("theme_mode") or "light"
       new = "dark" if mode == "light" else "light"
       db_set_setting("theme_mode", new)
       App.get_running_app().apply_theme()


   def on_snooze_change(self, spinner, text):
       db_set_setting("default_snooze", text)


   def on_default_alarm_change(self, spinner, text):
       db_set_setting("default_alarm", text)


   def refresh_sounds_list(self):
       self.sounds_list.clear_widgets()
       for fname in sorted(os.listdir(SOUNDS_DIR)):
           p = os.path.join(SOUNDS_DIR, fname)
           row = BoxLayout(size_hint_y=None, height=30)
           row.add_widget(Label(text=fname))
           del_btn = Button(text="Delete", size_hint_x=None, width=80)
           del_btn.bind(on_press=partial(self.delete_sound, p))
           row.add_widget(del_btn)
           self.sounds_list.add_widget(row)


   def delete_sound(self, path, *a):
       try:
           os.remove(path)
           self.refresh_sounds_list()
       except Exception as e:
           print("Delete sound error:", e)


   def reset_all_task_sounds(self, *a):
       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       c.execute("UPDATE tasks SET sound_path=NULL")
       conn.commit()
       conn.close()
       notification.notify(title="Reset", message="All tasks now use default ding")
       self.manager.reload_tasks_ui()




# ---------------------------
# Main manager
# ---------------------------
class TaskManagerAppRoot(BoxLayout):
   def __init__(self, **kwargs):
       super().__init__(orientation="vertical", **kwargs)
       # header
       header = BoxLayout(size_hint_y=None, height=50)
       header.add_widget(Label(text="🚀 Task Scheduler", size_hint_x=0.78))
       settings_btn = Button(text="⚙️", size_hint_x=0.1)
       settings_btn.bind(on_press=self.toggle_settings_overlay)
       header.add_widget(settings_btn)
       add_btn = Button(text="+ Add Task", size_hint_x=0.12)
       add_btn.bind(on_press=lambda *a: self.open_task_popup(mode="add"))
       header.add_widget(add_btn)
       self.add_widget(header)


       # body
       body = BoxLayout()
       left = BoxLayout(orientation="vertical")
       left.add_widget(Label(text="Active Tasks", size_hint_y=None, height=30))
       self.scroll = ScrollView()
       self.task_grid = GridLayout(cols=1, spacing=6, size_hint_y=None)
       self.task_grid.bind(minimum_height=self.task_grid.setter("height"))
       self.scroll.add_widget(self.task_grid)
       left.add_widget(self.scroll)
       body.add_widget(left)


       right = BoxLayout(orientation="vertical", size_hint_x=0.36)
       right.add_widget(Label(text="History", size_hint_y=None, height=30))
       self.history_scroll = ScrollView()
       self.history_grid = GridLayout(cols=1, spacing=6, size_hint_y=None)
       self.history_grid.bind(minimum_height=self.history_grid.setter("height"))
       self.history_scroll.add_widget(self.history_grid)
       right.add_widget(self.history_scroll)
       body.add_widget(right)
       self.add_widget(body)


       # settings overlay
       self.settings_overlay = SettingsOverlay(self)
       self.settings_open = False
       self.overlay_container = None


       # undo bar
       self.undo_bar = BoxLayout(size_hint_y=None, height=0)
       self.add_widget(self.undo_bar)


       # scheduler
       Clock.schedule_interval(self.check_due_tasks, CHECK_INTERVAL_SECONDS)
       self.reload_tasks_ui()


   def toggle_settings_overlay(self, *a):
       if not self.settings_open:
           self.overlay_container = BoxLayout(size_hint=(1, 1))
           bg = Button(background_color=(0, 0, 0, 0.4))
           bg.bind(on_press=self.toggle_settings_overlay)
           self.overlay_container.add_widget(bg)
           self.settings_overlay.refresh_sounds_list()
           self.overlay_container.add_widget(self.settings_overlay)
           self.add_widget(self.overlay_container)
           # slide in
           self.settings_overlay.x = Window.width
           Animation(x=Window.width * 0.2, duration=0.25).start(self.settings_overlay)
           self.settings_open = True
       else:
           Animation(x=Window.width, duration=0.25).bind(on_complete=lambda *a: self.remove_overlay()).start(self.settings_overlay)
           self.settings_open = False


   def remove_overlay(self, *a):
       try:
           self.remove_widget(self.overlay_container)
       except Exception:
           pass


   # open add/edit popup
   def open_task_popup(self, mode="add", task_id=None):
       task_data = None
       if mode in ("edit", "doagain") and task_id:
           conn = sqlite3.connect(DB_FILE)
           c = conn.cursor()
           c.execute("SELECT id, title, created_iso, due_iso, priority, alarm_type, sound_path, repeat_mode, repeat_days FROM tasks WHERE id=?", (task_id,))
           r = c.fetchone()
           conn.close()
           if r:
               task_data = {"id": r[0], "title": r[1], "created_iso": r[2], "due_iso": r[3], "priority": r[4], "alarm_type": r[5], "sound_path": r[6], "repeat_mode": r[7], "repeat_days": r[8]}
       popup = AddEditPopup(self, mode=mode, task_data=task_data)
       popup.open()


   def open_upload_default_sound(self, *a):
       # plyer.filechooser preferred, fallback to Kivy chooser
       try:
           selection = filechooser.open_file(title="Select default sound", filters=["*.mp3", "*.wav"])
           if selection and len(selection) > 0:
               src = selection[0]
               basename = f"default_{int(datetime.utcnow().timestamp())}_{os.path.basename(src)}"
               dest = os.path.join(SOUNDS_DIR, basename)
               with open(src, "rb") as rf, open(dest, "wb") as wf:
                   wf.write(rf.read())
               db_set_setting("global_default_sound", dest)
               notification.notify(title="Default sound set", message=os.path.basename(dest))
               self.settings_overlay.refresh_sounds_list()
       except Exception:
           # fallback chooser
           fc = FileChooserIconView(path=".", filters=["*.mp3","*.wav"])
           ok = Button(text="Select", size_hint_y=None, height=40)
           box = BoxLayout(orientation="vertical")
           box.add_widget(fc)
           row = BoxLayout(size_hint_y=None, height=40)
           row.add_widget(ok)
           close = Button(text="Close")
           row.add_widget(close)
           box.add_widget(row)
           popup = Popup(title="Choose default sound file", content=box, size_hint=(0.9, 0.9))
           ok.bind(on_press=lambda inst: self.set_default_sound(fc.selection, popup))
           close.bind(on_press=popup.dismiss)
           popup.open()


   def set_default_sound(self, selection, popup):
       if not selection: return
       src = selection[0]
       basename = f"default_{int(datetime.utcnow().timestamp())}_{os.path.basename(src)}"
       dest = os.path.join(SOUNDS_DIR, basename)
       try:
           with open(src, "rb") as rf, open(dest, "wb") as wf:
               wf.write(rf.read())
           db_set_setting("global_default_sound", dest)
           popup.dismiss()
           notification.notify(title="Default sound set", message=os.path.basename(dest))
       except Exception as e:
           print("set_default_sound error:", e)


   def reload_tasks_ui(self):
       self.task_grid.clear_widgets()
       self.history_grid.clear_widgets()
       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       c.execute("SELECT id, title, created_iso, due_iso, priority, alarm_type, sound_path, repeat_mode, repeat_days FROM tasks WHERE cancelled=0 AND completed_iso IS NULL ORDER BY CASE priority WHEN 'High' THEN 0 ELSE 1 END, id ASC")
       rows = c.fetchall()
       for r in rows:
           row = {"id": r[0], "title": r[1], "created_iso": r[2], "due_iso": r[3], "priority": r[4], "alarm_type": r[5], "sound_path": r[6], "repeat_mode": r[7], "repeat_days": r[8]}
           item = TaskItem(row, manager_ref=self)
           self.task_grid.add_widget(item)
       # history
       c.execute("SELECT id, title, completed_iso FROM tasks WHERE completed_iso IS NOT NULL ORDER BY completed_iso DESC")
       hist = c.fetchall()
       for r in hist:
           hid, title, completed_iso = r
           box = BoxLayout(size_hint_y=None, height=64)
           box.add_widget(Label(text=f"{title}\nDone: {friendly_dt(completed_iso)}"))
           btns = BoxLayout(size_hint_x=None, width=200)
           doagain = Button(text="Do Again", size_hint_x=None, width=100)
           doagain.bind(on_press=lambda inst, tid=hid: self.open_task_popup("doagain", tid))
           btns.add_widget(doagain)
           box.add_widget(btns)
           self.history_grid.add_widget(box)
       conn.close()


   def do_again_from_history(self, task_id):
       self.open_task_popup(mode="doagain", task_id=task_id)


   def confirm_cancel(self, task_id):
       p = Popup(title="Confirm Cancel", size_hint=(0.7, 0.4))
       b = BoxLayout(orientation="vertical")
       b.add_widget(Label(text="Are you sure you want to cancel this task?"))
       row = BoxLayout(size_hint_y=None, height=40)
       yes = Button(text="Yes", background_color=(1, 0.2, 0.2, 1))
       no = Button(text="No")
       row.add_widget(yes); row.add_widget(no); b.add_widget(row); p.content = b
       yes.bind(on_press=lambda *a: (self.cancel_task(task_id), p.dismiss()))
       no.bind(on_press=lambda *a: p.dismiss()); p.open()


   def cancel_task(self, task_id):
       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       c.execute("UPDATE tasks SET cancelled=1 WHERE id=?", (task_id,))
       conn.commit(); conn.close()
       self.reload_tasks_ui()
       self.show_undo("Task cancelled", lambda: self.undo_cancel(task_id))


   def complete_task(self, task_id):
       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       c.execute("SELECT repeat_mode, repeat_days, title, due_iso, priority, alarm_type, sound_path FROM tasks WHERE id=?", (task_id,))
       rr = c.fetchone()
       if not rr:
           conn.close(); return
       repeat_mode, repeat_days, title, due_iso, priority, alarm_type, sound_path = rr
       completed_iso = iso_now()
       c.execute("UPDATE tasks SET completed_iso=? WHERE id=?", (completed_iso, task_id))
       conn.commit(); conn.close()
       # handle repeating
       if repeat_mode and repeat_mode != "none" and repeat_mode != "once":
           try:
               due_dt = parse_iso(due_iso) or datetime.utcnow()
           except Exception:
               due_dt = datetime.utcnow()
           next_due = None
           if repeat_mode == "daily":
               next_due = due_dt + timedelta(days=1)
           elif repeat_mode == "weekly":
               days = [int(x) for x in (repeat_days or "").split(",") if x.strip().isdigit()]
               if days:
                   for delta in range(1, 14):
                       candidate = due_dt + timedelta(days=delta)
                       if candidate.weekday() in days:
                           next_due = candidate
                           break
               else:
                   next_due = due_dt + timedelta(days=7)
           elif repeat_mode == "forever":
               next_due = due_dt + timedelta(days=1)
           if next_due:
               nd_iso = next_due.isoformat()
               conn = sqlite3.connect(DB_FILE)
               c = conn.cursor()
               c.execute("""INSERT INTO tasks (title, created_iso, due_iso, priority, alarm_type, sound_path, repeat_mode, repeat_days)
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
                         (title, iso_now(), nd_iso, priority, alarm_type, sound_path, repeat_mode, repeat_days))
               conn.commit()
               conn.close()
       self.reload_tasks_ui()
       self.show_undo("Task completed", lambda: self.undo_complete(task_id))


   def undo_cancel(self, task_id):
       conn = sqlite3.connect(DB_FILE); c = conn.cursor()
       c.execute("UPDATE tasks SET cancelled=0 WHERE id=?", (task_id,)); conn.commit(); conn.close()
       self.reload_tasks_ui()


   def undo_complete(self, task_id):
       conn = sqlite3.connect(DB_FILE); c = conn.cursor()
       c.execute("UPDATE tasks SET completed_iso=NULL WHERE id=?", (task_id,)); conn.commit(); conn.close()
       self.reload_tasks_ui()


   def show_undo(self, message, undo_callback):
       self.undo_bar.clear_widgets(); self.undo_bar.height = 50
       self.undo_bar.add_widget(Label(text=message))
       ubtn = Button(text="Undo", size_hint_x=None, width=100)
       self.undo_bar.add_widget(ubtn)
       def finalize(dt=None):
           self.undo_bar.clear_widgets(); self.undo_bar.height = 0
       ubtn.bind(on_press=lambda *a: (undo_callback(), finalize()))
       secs = int(db_get_setting("undo_seconds") or UNDO_DEFAULT)
       Clock.schedule_once(lambda dt: finalize(), secs)


   def check_due_tasks(self, dt):
       now = datetime.utcnow()
       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       c.execute("SELECT id, title, due_iso, alarm_type, sound_path FROM tasks WHERE cancelled=0 AND completed_iso IS NULL")
       rows = c.fetchall(); conn.close()
       for r in rows:
           tid, title, due_iso, alarm_type, sound_path = r
           if not due_iso:
               continue
           try:
               due_dt = parse_iso(due_iso)
           except Exception:
               continue
           # trigger when due <= now and not more than e.g. 7 days late
           if due_dt and due_dt <= now and (now - due_dt) < timedelta(days=7):
               self.trigger_alarm(tid, title, alarm_type, sound_path)
               # bump due a bit to avoid immediate retrigger (simple guard)
               conn = sqlite3.connect(DB_FILE); c = conn.cursor()
               c.execute("UPDATE tasks SET due_iso=? WHERE id=?", ((due_dt + timedelta(seconds=60)).isoformat(), tid))
               conn.commit(); conn.close()


   def trigger_alarm(self, task_id, title, alarm_type, sound_path):
       # vibration if supported
       try:
           if alarm_type in ("ring_vibrate", "vibrate_notify"):
               vibrator.vibrate(0.5)
       except Exception:
           pass
       # determine sound path
       sound_to_use = sound_path or db_get_setting("global_default_sound") or ""
       if sound_to_use:
           SoundPlayer.play(sound_to_use)
       # notification (safe)
       try:
           notification.notify(title="Task Reminder", message=title)
       except Exception as e:
           print("Notification error:", e)
       # show popup with snooze/done
       p = Popup(title="Reminder", size_hint=(0.85, 0.4))
       bl = BoxLayout(orientation="vertical")
       bl.add_widget(Label(text=title))
       row = BoxLayout(size_hint_y=None, height=50)
       snooze = Button(text="Snooze")
       done = Button(text="Done", background_color=(0.2, 0.7, 0.2, 1))
       row.add_widget(snooze); row.add_widget(done)
       bl.add_widget(row); p.content = bl
       def do_done(*a):
           p.dismiss(); self.complete_task(task_id)
       def do_snooze(*a):
           p.dismiss(); mins = int(db_get_setting("default_snooze") or 5); self.snooze_task(task_id, mins)
       done.bind(on_press=do_done); snooze.bind(on_press=do_snooze)
       p.open()


   def snooze_task(self, task_id, minutes):
       conn = sqlite3.connect(DB_FILE); c = conn.cursor()
       c.execute("SELECT due_iso FROM tasks WHERE id=?", (task_id,)); r = c.fetchone()
       if not r:
           conn.close(); return
       try:
           dt = parse_iso(r[0]) or datetime.utcnow()
       except Exception:
           dt = datetime.utcnow()
       new_due = (dt + timedelta(minutes=minutes)).isoformat()
       c.execute("UPDATE tasks SET due_iso=? WHERE id=?", (new_due, task_id))
       conn.commit(); conn.close()
       notification.notify(title="Snoozed", message=f"Snoozed for {minutes} minutes")


   def start_async_sound(self, path):
       SoundPlayer.play(path)




# ---------------------------
# App class
# ---------------------------
class TaskSchedulerApp(App):
   def build(self):
       init_db()
       root = TaskManagerAppRoot()
       self.apply_theme()
       root.reload_tasks_ui()
       return root


   def apply_theme(self):
       mode = db_get_setting("theme_mode") or "light"
       if mode == "dark":
           Window.clearcolor = (0.08, 0.08, 0.08, 1)
       else:
           Window.clearcolor = (1, 1, 1, 1)




if __name__ == "__main__":
   TaskSchedulerApp().run()
"""
Universal Task Scheduler (single-file)
- Kivy UI
- SQLite local storage (reminders.db)
- Date/time picker popup
- Swipe gestures (left snooze, right done)
- Snooze, undo, repeat, history
- Per-task and global custom sounds (upload/preview/reset)
- Settings overlay slides from right
- Android-compatible plyer usage for notification and vibration
"""


import os
import sqlite3
import threading
from datetime import datetime, timedelta
from functools import partial


from kivy.app import App
from kivy.clock import Clock
from kivy.core.window import Window
from kivy.core.audio import SoundLoader
from kivy.properties import StringProperty, NumericProperty, BooleanProperty
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.gridlayout import GridLayout
from kivy.uix.scrollview import ScrollView
from kivy.uix.label import Label
from kivy.uix.button import Button
from kivy.uix.popup import Popup
from kivy.uix.spinner import Spinner
from kivy.uix.textinput import TextInput
from kivy.uix.filechooser import FileChooserIconView
from kivy.animation import Animation


# plyer features (notification, vibrator, filechooser)
from plyer import notification, vibrator, filechooser


# Constants
DB_FILE = "reminders.db"
SOUNDS_DIR = "sounds"
UNDO_DEFAULT = 10
HISTORY_CLEAN_DAYS = 30
CHECK_INTERVAL_SECONDS = 20  # how often scheduler checks due tasks


# Ensure sounds directory
os.makedirs(SOUNDS_DIR, exist_ok=True)




# ---------------------------
# Database helpers
# ---------------------------
def init_db():
   conn = sqlite3.connect(DB_FILE)
   c = conn.cursor()
   # tasks table
   c.execute("""
       CREATE TABLE IF NOT EXISTS tasks (
           id INTEGER PRIMARY KEY AUTOINCREMENT,
           title TEXT NOT NULL,
           created_iso TEXT NOT NULL,
           due_iso TEXT,
           priority TEXT DEFAULT 'Normal',
           alarm_type TEXT DEFAULT 'notify_ding',
           sound_path TEXT,
           repeat_mode TEXT DEFAULT 'none',
           repeat_days TEXT,
           completed_iso TEXT,
           cancelled INTEGER DEFAULT 0
       )
   """)
   # settings table
   c.execute("""
       CREATE TABLE IF NOT EXISTS settings (
           key TEXT PRIMARY KEY,
           value TEXT
       )
   """)
   # default settings
   defaults = {
       "global_default_sound": "",
       "default_alarm": "notify_ding",
       "default_snooze": "5",
       "theme_mode": "light",
       "undo_seconds": str(UNDO_DEFAULT)
   }
   for k, v in defaults.items():
       c.execute("INSERT OR IGNORE INTO settings (key, value) VALUES (?, ?)", (k, v))
   conn.commit()
   conn.close()


   # cleanup old history
   cleanup_history(HISTORY_CLEAN_DAYS)




def db_get_setting(key):
   conn = sqlite3.connect(DB_FILE)
   c = conn.cursor()
   c.execute("SELECT value FROM settings WHERE key=?", (key,))
   r = c.fetchone()
   conn.close()
   return r[0] if r else None




def db_set_setting(key, value):
   conn = sqlite3.connect(DB_FILE)
   c = conn.cursor()
   c.execute("INSERT OR REPLACE INTO settings (key, value) VALUES (?, ?)", (key, str(value)))
   conn.commit()
   conn.close()




def cleanup_history(days=30):
   cutoff = datetime.utcnow() - timedelta(days=days)
   cutoff_iso = cutoff.isoformat()
   conn = sqlite3.connect(DB_FILE)
   c = conn.cursor()
   try:
       c.execute("DELETE FROM tasks WHERE completed_iso IS NOT NULL AND completed_iso < ?", (cutoff_iso,))
       conn.commit()
   except Exception as e:
       print("cleanup_history error:", e)
   conn.close()




# ---------------------------
# Sound manager
# ---------------------------
class SoundPlayer:
   @staticmethod
   def play(path):
       if not path:
           # nothing to play
           return
       try:
           # use a thread so playback doesn't block UI
           def runner(p):
               try:
                   s = SoundLoader.load(p)
                   if s:
                       s.play()
               except Exception as e:
                   print("Sound play error:", e)
           t = threading.Thread(target=runner, args=(path,), daemon=True)
           t.start()
       except Exception as e:
           print("SoundPlayer error:", e)




# ---------------------------
# Small helpers
# ---------------------------
def iso_now():
   return datetime.utcnow().isoformat()




def parse_iso(s):
   try:
       return datetime.fromisoformat(s)
   except Exception:
       return None




def friendly_dt(iso):
   dt = parse_iso(iso)
   if not dt:
       return iso or ""
   return dt.strftime("%Y-%m-%d %H:%M")




# ---------------------------
# Task item (with swipe)
# ---------------------------
class TaskItem(BoxLayout):
   task_id = NumericProperty(0)
   title = StringProperty("")
   due_iso = StringProperty("")
   priority = StringProperty("Normal")
   completed = BooleanProperty(False)


   def __init__(self, row, manager_ref, **kwargs):
       super().__init__(orientation="horizontal", size_hint_y=None, height=64, padding=6, **kwargs)
       self.manager = manager_ref
       self.task_id = row.get("id")
       self.title = row.get("title")
       self.due_iso = row.get("due_iso") or ""
       self.priority = row.get("priority") or "Normal"
       self.alarm_type = row.get("alarm_type")
       self.sound_path = row.get("sound_path")
       self.repeat_mode = row.get("repeat_mode")
       self.repeat_days = row.get("repeat_days")
       self.completed = bool(row.get("completed_iso"))
       self.cancelled = bool(row.get("cancelled"))


       # label area
       left = BoxLayout(orientation="vertical")
       pr_symbol = "🔴 " if self.priority == "High" else ""
       left.add_widget(Label(text=f"{pr_symbol}{self.title}", halign="left", valign="middle"))
       left.add_widget(Label(text=f"Due: {friendly_dt(self.due_iso)}", font_size=12, halign="left"))
       self.add_widget(left)


       # action buttons
       btns = BoxLayout(size_hint_x=None, width=260, spacing=6)
       edit_btn = Button(text="Edit", size_hint_x=None, width=70)
       edit_btn.bind(on_press=self.on_edit)
       btns.add_widget(edit_btn)


       done_btn = Button(text="Done", size_hint_x=None, width=60)
       done_btn.bind(on_press=self.on_complete)
       btns.add_widget(done_btn)


       snooze_btn = Button(text="Snooze", size_hint_x=None, width=80)
       snooze_btn.bind(on_press=lambda *a: self.manager.snooze_task(self.task_id, int(db_get_setting("default_snooze") or 5)))
       btns.add_widget(snooze_btn)


       cancel_btn = Button(text="Cancel", size_hint_x=None, width=60, background_color=(1, .3, .3, 1))
       cancel_btn.bind(on_press=self.on_cancel)
       btns.add_widget(cancel_btn)


       self.add_widget(btns)


       self._touch_start_x = None


   def on_edit(self, *a):
       self.manager.open_task_popup(mode="edit", task_id=self.task_id)


   def on_complete(self, *a):
       self.manager.complete_task(self.task_id)


   def on_cancel(self, *a):
       self.manager.confirm_cancel(self.task_id)


   def on_touch_down(self, touch):
       if self.collide_point(*touch.pos):
           self._touch_start_x = touch.x
       return super().on_touch_down(touch)


   def on_touch_up(self, touch):
       if self._touch_start_x is None:
           return super().on_touch_up(touch)
       dx = touch.x - self._touch_start_x
       self._touch_start_x = None
       if abs(dx) < 50:
           return super().on_touch_up(touch)
       if dx < 0:
           # swipe left -> snooze
           try:
               snooze_min = int(db_get_setting("default_snooze") or 5)
               self.manager.snooze_task(self.task_id, snooze_min)
           except Exception as e:
               print("snooze error:", e)
       else:
           # swipe right -> complete
           self.manager.complete_task(self.task_id)
       return True




# ---------------------------
# Add/Edit Task Popup
# ---------------------------
class AddEditPopup(Popup):
   def __init__(self, manager_ref, mode="add", task_data=None, **kwargs):
       title = "Add Task" if mode == "add" else "Edit Task" if mode == "edit" else "Do Again"
       super().__init__(title=title, size_hint=(0.96, 0.9), **kwargs)
       self.manager = manager_ref
       self.mode = mode
       self.task_data = task_data or {}
       layout = BoxLayout(orientation="vertical", spacing=8, padding=8)


       # Title
       self.title_input = TextInput(text=self.task_data.get("title", ""), size_hint_y=None, height=40, hint_text="Task title")
       layout.add_widget(self.title_input)


       # Date/time spinners
       dt_box = BoxLayout(size_hint_y=None, height=40)
       now = datetime.now()
       years = [str(y) for y in range(now.year, now.year + 3)]
       months = [str(m).zfill(2) for m in range(1, 13)]
       days = [str(d).zfill(2) for d in range(1, 32)]
       self.year = Spinner(text=str(now.year), values=years, size_hint_x=None, width=100)
       self.month = Spinner(text=str(now.month).zfill(2), values=months, size_hint_x=None, width=80)
       self.day = Spinner(text=str(now.day).zfill(2), values=days, size_hint_x=None, width=80)
       hours = [str(h).zfill(2) for h in range(0, 24)]
       mins = [str(m).zfill(2) for m in range(0, 60, 5)]
       self.hour = Spinner(text=str(now.hour).zfill(2), values=hours, size_hint_x=None, width=80)
       self.minute = Spinner(text=str((now.minute//5)*5).zfill(2), values=mins, size_hint_x=None, width=80)
       dt_box.add_widget(self.year); dt_box.add_widget(self.month); dt_box.add_widget(self.day)
       dt_box.add_widget(self.hour); dt_box.add_widget(self.minute)
       layout.add_widget(dt_box)


       # Priority
       prio_box = BoxLayout(size_hint_y=None, height=40)
       self.prio_spinner = Spinner(text=self.task_data.get("priority", db_get_setting("default_priority") or "Normal"), values=["Normal", "High"], size_hint_x=None, width=140)
       prio_box.add_widget(Label(text="Priority:", size_hint_x=None, width=80))
       prio_box.add_widget(self.prio_spinner)
       layout.add_widget(prio_box)


       # Alarm type
       alarm_box = BoxLayout(size_hint_y=None, height=40)
       self.alarm_spinner = Spinner(text=self.task_data.get("alarm_type", db_get_setting("default_alarm") or "notify_ding"),
                                    values=["notify_ding", "ring_vibrate", "vibrate_notify"], size_hint_x=None, width=220)
       alarm_box.add_widget(Label(text="Alarm:", size_hint_x=None, width=80))
       alarm_box.add_widget(self.alarm_spinner)
       layout.add_widget(alarm_box)


       # Sound upload + preview + reset
       sound_box = BoxLayout(size_hint_y=None, height=40)
       self.sound_label = Label(text=os.path.basename(self.task_data.get("sound_path")) if self.task_data.get("sound_path") else "Default (ding)", halign="left")
       sound_box.add_widget(self.sound_label)
       upload_btn = Button(text="Upload Sound", size_hint_x=None, width=120)
       upload_btn.bind(on_press=self.open_filechooser)
       sound_box.add_widget(upload_btn)
       preview_btn = Button(text="Preview", size_hint_x=None, width=80)
       preview_btn.bind(on_press=self.preview_sound)
       sound_box.add_widget(preview_btn)
       reset_btn = Button(text="Reset", size_hint_x=None, width=80)
       reset_btn.bind(on_press=self.reset_to_default_sound)
       sound_box.add_widget(reset_btn)
       layout.add_widget(sound_box)


       # Repeat
       repeat_box = BoxLayout(size_hint_y=None, height=40)
       self.repeat_spinner = Spinner(text=self.task_data.get("repeat_mode", "none"), values=["none", "once", "daily", "weekly", "forever"], size_hint_x=None, width=140)
       repeat_box.add_widget(Label(text="Repeat:", size_hint_x=None, width=80))
       repeat_box.add_widget(self.repeat_spinner)
       layout.add_widget(repeat_box)


       # Weekdays toggles for weekly repeat
       wd_box = BoxLayout(size_hint_y=None, height=40)
       self.weekday_buttons = []
       for i, wd in enumerate(["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]):
           b = Button(text=wd, size_hint_x=None, width=44)
           b.background_color = (0.85, 0.85, 0.85, 1)
           b.bind(on_press=partial(self.toggle_weekday, i))
           self.weekday_buttons.append(b)
           wd_box.add_widget(b)
       layout.add_widget(wd_box)


       # Footer (cancel/save)
       footer = BoxLayout(size_hint_y=None, height=50)
       cancel_btn = Button(text="Cancel")
       cancel_btn.bind(on_press=self.dismiss)
       save_btn = Button(text="Save", background_color=(0.2, 0.7, 0.2, 1))
       save_btn.bind(on_press=self.on_save)
       footer.add_widget(cancel_btn)
       footer.add_widget(save_btn)
       layout.add_widget(footer)


       # populate fields if editing / doagain
       if self.mode in ("edit", "doagain") and self.task_data:
           try:
               dt = parse_iso(self.task_data.get("due_iso"))
               if dt:
                   self.year.text = str(dt.year)
                   self.month.text = str(dt.month).zfill(2)
                   self.day.text = str(dt.day).zfill(2)
                   self.hour.text = str(dt.hour).zfill(2)
                   self.minute.text = str(dt.minute).zfill(2)
           except Exception:
               pass
           self.title_input.text = self.task_data.get("title", "")
           self.prio_spinner.text = self.task_data.get("priority", "Normal")
           self.alarm_spinner.text = self.task_data.get("alarm_type", db_get_setting("default_alarm") or "notify_ding")
           sp = self.task_data.get("sound_path")
           if sp:
               self.sound_label.text = os.path.basename(sp)
               self._selected_sound = sp
           else:
               self.sound_label.text = "Default (ding)"
               self._selected_sound = None
           self.repeat_spinner.text = self.task_data.get("repeat_mode", "none")
           if self.task_data.get("repeat_days"):
               for idx_str in (self.task_data.get("repeat_days") or "").split(","):
                   if idx_str.strip().isdigit():
                       idx = int(idx_str)
                       if 0 <= idx < 7:
                           self.weekday_buttons[idx].background_color = (0.45, 0.85, 0.45, 1)
       else:
           self._selected_sound = None


       self.content = layout
       self.filechooser_popup = None


   def toggle_weekday(self, idx, *a):
       b = self.weekday_buttons[idx]
       if b.background_color[0] > 0.7:
           b.background_color = (0.45, 0.85, 0.45, 1)
       else:
           b.background_color = (0.85, 0.85, 0.85, 1)


   def open_filechooser(self, *a):
       # Try using plyer.filechooser first (Android-friendly)
       try:
           selection = filechooser.open_file(title="Select sound file", filters=["*.wav", "*.mp3"])
           if selection and len(selection) > 0:
               src = selection[0]
               basename = f"{int(datetime.utcnow().timestamp())}_{os.path.basename(src)}"
               dest = os.path.join(SOUNDS_DIR, basename)
               try:
                   with open(src, "rb") as rf, open(dest, "wb") as wf:
                       wf.write(rf.read())
                   self._selected_sound = dest
                   self.sound_label.text = os.path.basename(dest)
               except Exception as e:
                   print("Copy error:", e)
       except Exception:
           # fallback to Kivy filechooser popup
           if not self.filechooser_popup:
               fc = FileChooserIconView(path=".", filters=["*.mp3", "*.wav"])
               ok = Button(text="Select", size_hint_y=None, height=40)
               box = BoxLayout(orientation="vertical")
               box.add_widget(fc)
               row = BoxLayout(size_hint_y=None, height=40)
               row.add_widget(ok)
               close = Button(text="Close")
               row.add_widget(close)
               box.add_widget(row)
               popup = Popup(title="Choose a sound file", content=box, size_hint=(0.9, 0.9))
               ok.bind(on_press=lambda inst: self.select_file(fc.path, fc.selection, popup))
               close.bind(on_press=popup.dismiss)
               self.filechooser_popup = popup
           self.filechooser_popup.open()


   def select_file(self, path, selection, popup):
       if not selection:
           return
       src = selection[0]
       basename = f"{int(datetime.utcnow().timestamp())}_{os.path.basename(src)}"
       dest = os.path.join(SOUNDS_DIR, basename)
       try:
           with open(src, "rb") as rf, open(dest, "wb") as wf:
               wf.write(rf.read())
           self._selected_sound = dest
           self.sound_label.text = os.path.basename(dest)
           popup.dismiss()
       except Exception as e:
           print("File copy error:", e)


   def preview_sound(self, *a):
       path = getattr(self, "_selected_sound", None)
       if not path:
           global_def = db_get_setting("global_default_sound") or ""
           if global_def:
               SoundPlayer.play(global_def)
           else:
               notification.notify(title="Preview", message="Default ding (system)")
       else:
           SoundPlayer.play(path)


   def reset_to_default_sound(self, *a):
       self._selected_sound = None
       self.sound_label.text = "Default (ding)"


   def on_save(self, *a):
       # build due iso
       try:
           yr = int(self.year.text); mo = int(self.month.text); da = int(self.day.text)
           hr = int(self.hour.text); mn = int(self.minute.text)
           due = datetime(year=yr, month=mo, day=da, hour=hr, minute=mn)
           due_iso = due.isoformat()
       except Exception:
           notification.notify(title="Error", message="Invalid date/time")
           return
       title = self.title_input.text.strip()
       if not title:
           notification.notify(title="Error", message="Please enter a title")
           return
       pr = self.prio_spinner.text
       alarm = self.alarm_spinner.text
       soundp = getattr(self, "_selected_sound", None)
       repeat_mode = self.repeat_spinner.text
       repeat_days = ",".join(str(i) for i,b in enumerate(self.weekday_buttons) if b.background_color[0] < 0.7)


       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       if self.mode == "edit" and self.task_data and self.task_data.get("id"):
           c.execute("""UPDATE tasks SET title=?, due_iso=?, priority=?, alarm_type=?, sound_path=?, repeat_mode=?, repeat_days=? WHERE id=?""",
                     (title, due_iso, pr, alarm, soundp, repeat_mode, repeat_days, self.task_data["id"]))
       else:
           created = iso_now()
           c.execute("""INSERT INTO tasks (title, created_iso, due_iso, priority, alarm_type, sound_path, repeat_mode, repeat_days)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
                     (title, created, due_iso, pr, alarm, soundp, repeat_mode, repeat_days))
       conn.commit()
       conn.close()
       self.manager.reload_tasks_ui()
       self.dismiss()




# ---------------------------
# Settings overlay (slides from right)
# ---------------------------
class SettingsOverlay(BoxLayout):
   def __init__(self, manager_ref, **kwargs):
       super().__init__(orientation="vertical", size_hint=(0.8, 1), **kwargs)
       self.manager = manager_ref
       self.padding = 10
       self.spacing = 8
       self.add_widget(Label(text="Settings", size_hint_y=None, height=40))
       # theme toggle
       theme_box = BoxLayout(size_hint_y=None, height=40)
       theme_box.add_widget(Label(text="Theme:", size_hint_x=None, width=80))
       self.theme_btn = Button(text="Toggle Light/Dark")
       self.theme_btn.bind(on_press=self.toggle_theme)
       theme_box.add_widget(self.theme_btn)
       self.add_widget(theme_box)
       # snooze default
       snooze_box = BoxLayout(size_hint_y=None, height=40)
       snooze_box.add_widget(Label(text="Default Snooze (min):", size_hint_x=None, width=150))
       self.snooze_spinner = Spinner(text=db_get_setting("default_snooze") or "5", values=[str(v) for v in (5,10,15,20)], size_hint_x=None, width=80)
       self.snooze_spinner.bind(text=self.on_snooze_change)
       snooze_box.add_widget(self.snooze_spinner)
       self.add_widget(snooze_box)
       # default alarm
       alarm_box = BoxLayout(size_hint_y=None, height=40)
       alarm_box.add_widget(Label(text="Default Alarm:", size_hint_x=None, width=120))
       self.alarm_spinner = Spinner(text=db_get_setting("default_alarm") or "notify_ding",
                                    values=["notify_ding","ring_vibrate","vibrate_notify"], size_hint_x=None, width=180)
       self.alarm_spinner.bind(text=self.on_default_alarm_change)
       alarm_box.add_widget(self.alarm_spinner)
       self.add_widget(alarm_box)
       # sounds management
       self.add_widget(Label(text="Uploaded Sounds:", size_hint_y=None, height=30))
       self.sounds_list = BoxLayout(orientation="vertical")
       self.refresh_sounds_list()
       self.add_widget(self.sounds_list)
       upload_global = Button(text="Upload Default Sound")
       upload_global.bind(on_press=manager_ref.open_upload_default_sound)
       self.add_widget(upload_global)
       reset_all_btn = Button(text="Reset All Task Sounds to Ding")
       reset_all_btn.bind(on_press=self.reset_all_task_sounds)
       self.add_widget(reset_all_btn)
       close = Button(text="Close", size_hint_y=None, height=40)
       close.bind(on_press=manager_ref.toggle_settings_overlay)
       self.add_widget(close)


   def toggle_theme(self, *a):
       mode = db_get_setting("theme_mode") or "light"
       new = "dark" if mode == "light" else "light"
       db_set_setting("theme_mode", new)
       App.get_running_app().apply_theme()


   def on_snooze_change(self, spinner, text):
       db_set_setting("default_snooze", text)


   def on_default_alarm_change(self, spinner, text):
       db_set_setting("default_alarm", text)


   def refresh_sounds_list(self):
       self.sounds_list.clear_widgets()
       for fname in sorted(os.listdir(SOUNDS_DIR)):
           p = os.path.join(SOUNDS_DIR, fname)
           row = BoxLayout(size_hint_y=None, height=30)
           row.add_widget(Label(text=fname))
           del_btn = Button(text="Delete", size_hint_x=None, width=80)
           del_btn.bind(on_press=partial(self.delete_sound, p))
           row.add_widget(del_btn)
           self.sounds_list.add_widget(row)


   def delete_sound(self, path, *a):
       try:
           os.remove(path)
           self.refresh_sounds_list()
       except Exception as e:
           print("Delete sound error:", e)


   def reset_all_task_sounds(self, *a):
       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       c.execute("UPDATE tasks SET sound_path=NULL")
       conn.commit()
       conn.close()
       notification.notify(title="Reset", message="All tasks now use default ding")
       self.manager.reload_tasks_ui()




# ---------------------------
# Main manager
# ---------------------------
class TaskManagerAppRoot(BoxLayout):
   def __init__(self, **kwargs):
       super().__init__(orientation="vertical", **kwargs)
       # header
       header = BoxLayout(size_hint_y=None, height=50)
       header.add_widget(Label(text="🚀 Task Scheduler", size_hint_x=0.78))
       settings_btn = Button(text="⚙️", size_hint_x=0.1)
       settings_btn.bind(on_press=self.toggle_settings_overlay)
       header.add_widget(settings_btn)
       add_btn = Button(text="+ Add Task", size_hint_x=0.12)
       add_btn.bind(on_press=lambda *a: self.open_task_popup(mode="add"))
       header.add_widget(add_btn)
       self.add_widget(header)


       # body
       body = BoxLayout()
       left = BoxLayout(orientation="vertical")
       left.add_widget(Label(text="Active Tasks", size_hint_y=None, height=30))
       self.scroll = ScrollView()
       self.task_grid = GridLayout(cols=1, spacing=6, size_hint_y=None)
       self.task_grid.bind(minimum_height=self.task_grid.setter("height"))
       self.scroll.add_widget(self.task_grid)
       left.add_widget(self.scroll)
       body.add_widget(left)


       right = BoxLayout(orientation="vertical", size_hint_x=0.36)
       right.add_widget(Label(text="History", size_hint_y=None, height=30))
       self.history_scroll = ScrollView()
       self.history_grid = GridLayout(cols=1, spacing=6, size_hint_y=None)
       self.history_grid.bind(minimum_height=self.history_grid.setter("height"))
       self.history_scroll.add_widget(self.history_grid)
       right.add_widget(self.history_scroll)
       body.add_widget(right)
       self.add_widget(body)


       # settings overlay
       self.settings_overlay = SettingsOverlay(self)
       self.settings_open = False
       self.overlay_container = None


       # undo bar
       self.undo_bar = BoxLayout(size_hint_y=None, height=0)
       self.add_widget(self.undo_bar)


       # scheduler
       Clock.schedule_interval(self.check_due_tasks, CHECK_INTERVAL_SECONDS)
       self.reload_tasks_ui()


   def toggle_settings_overlay(self, *a):
       if not self.settings_open:
           self.overlay_container = BoxLayout(size_hint=(1, 1))
           bg = Button(background_color=(0, 0, 0, 0.4))
           bg.bind(on_press=self.toggle_settings_overlay)
           self.overlay_container.add_widget(bg)
           self.settings_overlay.refresh_sounds_list()
           self.overlay_container.add_widget(self.settings_overlay)
           self.add_widget(self.overlay_container)
           # slide in
           self.settings_overlay.x = Window.width
           Animation(x=Window.width * 0.2, duration=0.25).start(self.settings_overlay)
           self.settings_open = True
       else:
           Animation(x=Window.width, duration=0.25).bind(on_complete=lambda *a: self.remove_overlay()).start(self.settings_overlay)
           self.settings_open = False


   def remove_overlay(self, *a):
       try:
           self.remove_widget(self.overlay_container)
       except Exception:
           pass


   # open add/edit popup
   def open_task_popup(self, mode="add", task_id=None):
       task_data = None
       if mode in ("edit", "doagain") and task_id:
           conn = sqlite3.connect(DB_FILE)
           c = conn.cursor()
           c.execute("SELECT id, title, created_iso, due_iso, priority, alarm_type, sound_path, repeat_mode, repeat_days FROM tasks WHERE id=?", (task_id,))
           r = c.fetchone()
           conn.close()
           if r:
               task_data = {"id": r[0], "title": r[1], "created_iso": r[2], "due_iso": r[3], "priority": r[4], "alarm_type": r[5], "sound_path": r[6], "repeat_mode": r[7], "repeat_days": r[8]}
       popup = AddEditPopup(self, mode=mode, task_data=task_data)
       popup.open()


   def open_upload_default_sound(self, *a):
       # plyer.filechooser preferred, fallback to Kivy chooser
       try:
           selection = filechooser.open_file(title="Select default sound", filters=["*.mp3", "*.wav"])
           if selection and len(selection) > 0:
               src = selection[0]
               basename = f"default_{int(datetime.utcnow().timestamp())}_{os.path.basename(src)}"
               dest = os.path.join(SOUNDS_DIR, basename)
               with open(src, "rb") as rf, open(dest, "wb") as wf:
                   wf.write(rf.read())
               db_set_setting("global_default_sound", dest)
               notification.notify(title="Default sound set", message=os.path.basename(dest))
               self.settings_overlay.refresh_sounds_list()
       except Exception:
           # fallback chooser
           fc = FileChooserIconView(path=".", filters=["*.mp3","*.wav"])
           ok = Button(text="Select", size_hint_y=None, height=40)
           box = BoxLayout(orientation="vertical")
           box.add_widget(fc)
           row = BoxLayout(size_hint_y=None, height=40)
           row.add_widget(ok)
           close = Button(text="Close")
           row.add_widget(close)
           box.add_widget(row)
           popup = Popup(title="Choose default sound file", content=box, size_hint=(0.9, 0.9))
           ok.bind(on_press=lambda inst: self.set_default_sound(fc.selection, popup))
           close.bind(on_press=popup.dismiss)
           popup.open()


   def set_default_sound(self, selection, popup):
       if not selection: return
       src = selection[0]
       basename = f"default_{int(datetime.utcnow().timestamp())}_{os.path.basename(src)}"
       dest = os.path.join(SOUNDS_DIR, basename)
       try:
           with open(src, "rb") as rf, open(dest, "wb") as wf:
               wf.write(rf.read())
           db_set_setting("global_default_sound", dest)
           popup.dismiss()
           notification.notify(title="Default sound set", message=os.path.basename(dest))
       except Exception as e:
           print("set_default_sound error:", e)


   def reload_tasks_ui(self):
       self.task_grid.clear_widgets()
       self.history_grid.clear_widgets()
       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       c.execute("SELECT id, title, created_iso, due_iso, priority, alarm_type, sound_path, repeat_mode, repeat_days FROM tasks WHERE cancelled=0 AND completed_iso IS NULL ORDER BY CASE priority WHEN 'High' THEN 0 ELSE 1 END, id ASC")
       rows = c.fetchall()
       for r in rows:
           row = {"id": r[0], "title": r[1], "created_iso": r[2], "due_iso": r[3], "priority": r[4], "alarm_type": r[5], "sound_path": r[6], "repeat_mode": r[7], "repeat_days": r[8]}
           item = TaskItem(row, manager_ref=self)
           self.task_grid.add_widget(item)
       # history
       c.execute("SELECT id, title, completed_iso FROM tasks WHERE completed_iso IS NOT NULL ORDER BY completed_iso DESC")
       hist = c.fetchall()
       for r in hist:
           hid, title, completed_iso = r
           box = BoxLayout(size_hint_y=None, height=64)
           box.add_widget(Label(text=f"{title}\nDone: {friendly_dt(completed_iso)}"))
           btns = BoxLayout(size_hint_x=None, width=200)
           doagain = Button(text="Do Again", size_hint_x=None, width=100)
           doagain.bind(on_press=lambda inst, tid=hid: self.open_task_popup("doagain", tid))
           btns.add_widget(doagain)
           box.add_widget(btns)
           self.history_grid.add_widget(box)
       conn.close()


   def do_again_from_history(self, task_id):
       self.open_task_popup(mode="doagain", task_id=task_id)


   def confirm_cancel(self, task_id):
       p = Popup(title="Confirm Cancel", size_hint=(0.7, 0.4))
       b = BoxLayout(orientation="vertical")
       b.add_widget(Label(text="Are you sure you want to cancel this task?"))
       row = BoxLayout(size_hint_y=None, height=40)
       yes = Button(text="Yes", background_color=(1, 0.2, 0.2, 1))
       no = Button(text="No")
       row.add_widget(yes); row.add_widget(no); b.add_widget(row); p.content = b
       yes.bind(on_press=lambda *a: (self.cancel_task(task_id), p.dismiss()))
       no.bind(on_press=lambda *a: p.dismiss()); p.open()


   def cancel_task(self, task_id):
       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       c.execute("UPDATE tasks SET cancelled=1 WHERE id=?", (task_id,))
       conn.commit(); conn.close()
       self.reload_tasks_ui()
       self.show_undo("Task cancelled", lambda: self.undo_cancel(task_id))


   def complete_task(self, task_id):
       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       c.execute("SELECT repeat_mode, repeat_days, title, due_iso, priority, alarm_type, sound_path FROM tasks WHERE id=?", (task_id,))
       rr = c.fetchone()
       if not rr:
           conn.close(); return
       repeat_mode, repeat_days, title, due_iso, priority, alarm_type, sound_path = rr
       completed_iso = iso_now()
       c.execute("UPDATE tasks SET completed_iso=? WHERE id=?", (completed_iso, task_id))
       conn.commit(); conn.close()
       # handle repeating
       if repeat_mode and repeat_mode != "none" and repeat_mode != "once":
           try:
               due_dt = parse_iso(due_iso) or datetime.utcnow()
           except Exception:
               due_dt = datetime.utcnow()
           next_due = None
           if repeat_mode == "daily":
               next_due = due_dt + timedelta(days=1)
           elif repeat_mode == "weekly":
               days = [int(x) for x in (repeat_days or "").split(",") if x.strip().isdigit()]
               if days:
                   for delta in range(1, 14):
                       candidate = due_dt + timedelta(days=delta)
                       if candidate.weekday() in days:
                           next_due = candidate
                           break
               else:
                   next_due = due_dt + timedelta(days=7)
           elif repeat_mode == "forever":
               next_due = due_dt + timedelta(days=1)
           if next_due:
               nd_iso = next_due.isoformat()
               conn = sqlite3.connect(DB_FILE)
               c = conn.cursor()
               c.execute("""INSERT INTO tasks (title, created_iso, due_iso, priority, alarm_type, sound_path, repeat_mode, repeat_days)
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
                         (title, iso_now(), nd_iso, priority, alarm_type, sound_path, repeat_mode, repeat_days))
               conn.commit()
               conn.close()
       self.reload_tasks_ui()
       self.show_undo("Task completed", lambda: self.undo_complete(task_id))


   def undo_cancel(self, task_id):
       conn = sqlite3.connect(DB_FILE); c = conn.cursor()
       c.execute("UPDATE tasks SET cancelled=0 WHERE id=?", (task_id,)); conn.commit(); conn.close()
       self.reload_tasks_ui()


   def undo_complete(self, task_id):
       conn = sqlite3.connect(DB_FILE); c = conn.cursor()
       c.execute("UPDATE tasks SET completed_iso=NULL WHERE id=?", (task_id,)); conn.commit(); conn.close()
       self.reload_tasks_ui()


   def show_undo(self, message, undo_callback):
       self.undo_bar.clear_widgets(); self.undo_bar.height = 50
       self.undo_bar.add_widget(Label(text=message))
       ubtn = Button(text="Undo", size_hint_x=None, width=100)
       self.undo_bar.add_widget(ubtn)
       def finalize(dt=None):
           self.undo_bar.clear_widgets(); self.undo_bar.height = 0
       ubtn.bind(on_press=lambda *a: (undo_callback(), finalize()))
       secs = int(db_get_setting("undo_seconds") or UNDO_DEFAULT)
       Clock.schedule_once(lambda dt: finalize(), secs)


   def check_due_tasks(self, dt):
       now = datetime.utcnow()
       conn = sqlite3.connect(DB_FILE)
       c = conn.cursor()
       c.execute("SELECT id, title, due_iso, alarm_type, sound_path FROM tasks WHERE cancelled=0 AND completed_iso IS NULL")
       rows = c.fetchall(); conn.close()
       for r in rows:
           tid, title, due_iso, alarm_type, sound_path = r
           if not due_iso:
               continue
           try:
               due_dt = parse_iso(due_iso)
           except Exception:
               continue
           # trigger when due <= now and not more than e.g. 7 days late
           if due_dt and due_dt <= now and (now - due_dt) < timedelta(days=7):
               self.trigger_alarm(tid, title, alarm_type, sound_path)
               # bump due a bit to avoid immediate retrigger (simple guard)
               conn = sqlite3.connect(DB_FILE); c = conn.cursor()
               c.execute("UPDATE tasks SET due_iso=? WHERE id=?", ((due_dt + timedelta(seconds=60)).isoformat(), tid))
               conn.commit(); conn.close()


   def trigger_alarm(self, task_id, title, alarm_type, sound_path):
       # vibration if supported
       try:
           if alarm_type in ("ring_vibrate", "vibrate_notify"):
               vibrator.vibrate(0.5)
       except Exception:
           pass
       # determine sound path
       sound_to_use = sound_path or db_get_setting("global_default_sound") or ""
       if sound_to_use:
           SoundPlayer.play(sound_to_use)
       # notification (safe)
       try:
           notification.notify(title="Task Reminder", message=title)
       except Exception as e:
           print("Notification error:", e)
       # show popup with snooze/done
       p = Popup(title="Reminder", size_hint=(0.85, 0.4))
       bl = BoxLayout(orientation="vertical")
       bl.add_widget(Label(text=title))
       row = BoxLayout(size_hint_y=None, height=50)
       snooze = Button(text="Snooze")
       done = Button(text="Done", background_color=(0.2, 0.7, 0.2, 1))
       row.add_widget(snooze); row.add_widget(done)
       bl.add_widget(row); p.content = bl
       def do_done(*a):
           p.dismiss(); self.complete_task(task_id)
       def do_snooze(*a):
           p.dismiss(); mins = int(db_get_setting("default_snooze") or 5); self.snooze_task(task_id, mins)
       done.bind(on_press=do_done); snooze.bind(on_press=do_snooze)
       p.open()


   def snooze_task(self, task_id, minutes):
       conn = sqlite3.connect(DB_FILE); c = conn.cursor()
       c.execute("SELECT due_iso FROM tasks WHERE id=?", (task_id,)); r = c.fetchone()
       if not r:
           conn.close(); return
       try:
           dt = parse_iso(r[0]) or datetime.utcnow()
       except Exception:
           dt = datetime.utcnow()
       new_due = (dt + timedelta(minutes=minutes)).isoformat()
       c.execute("UPDATE tasks SET due_iso=? WHERE id=?", (new_due, task_id))
       conn.commit(); conn.close()
       notification.notify(title="Snoozed", message=f"Snoozed for {minutes} minutes")


   def start_async_sound(self, path):
       SoundPlayer.play(path)




# ---------------------------
# App class
# ---------------------------
class TaskSchedulerApp(App):
   def build(self):
       init_db()
       root = TaskManagerAppRoot()
       self.apply_theme()
       root.reload_tasks_ui()
       return root


   def apply_theme(self):
       mode = db_get_setting("theme_mode") or "light"
       if mode == "dark":
           Window.clearcolor = (0.08, 0.08, 0.08, 1)
       else:
           Window.clearcolor = (1, 1, 1, 1)




if __name__ == "__main__":
   TaskSchedulerApp().run()

